# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [6]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
# lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
# lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lats = np.random.uniform(lat_range[0], lat_range[1], size=30)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=30)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
display(len(cities))
display(cities)

28

['santa maria',
 'tiksi',
 'rikitea',
 'jamestown',
 'campbell river',
 'ushuaia',
 'karpathos',
 'illoqqortoormiut',
 'phan rang',
 'mahebourg',
 'tasiilaq',
 'vaini',
 'mataura',
 'sitka',
 'victor harbor',
 'nizhneyansk',
 'puerto ayora',
 'chicama',
 'rocha',
 'avarua',
 'castro',
 'tsihombe',
 'lagoa',
 'atuona',
 'thompson',
 'cururupu',
 'mount gambier',
 'albany']

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [32]:
from pprint import pprint

# Check for a single city
city = cities[0]

request_url = "https://api.openweathermap.org/data/2.5/weather?q=%s&appid=%s" % (city, weather_api_key)
response = requests.get(request_url)
pprint(response.json())

{'base': 'stations',
 'clouds': {'all': 0},
 'cod': 200,
 'coord': {'lat': -29.68, 'lon': -53.81},
 'dt': 1609463790,
 'id': 3450083,
 'main': {'feels_like': 292.76,
          'humidity': 56,
          'pressure': 1013,
          'temp': 293.15,
          'temp_max': 293.15,
          'temp_min': 293.15},
 'name': 'Santa Maria',
 'sys': {'country': 'BR',
         'id': 8430,
         'sunrise': 1609403872,
         'sunset': 1609454325,
         'type': 1},
 'timezone': -10800,
 'visibility': 10000,
 'weather': [{'description': 'clear sky',
              'icon': '01n',
              'id': 800,
              'main': 'Clear'}],
 'wind': {'deg': 280, 'speed': 1}}


In [33]:
response.raise_for_status()

### Handling limited API requests
- The number of requests from openweather api is limited to 60 requests per minute. To stay inside this limit
1. Divide the cities into sets of 60
2. Request api for every 1 second using time.sleep(1)
3. Make sure that the requests are limited to 1 set per minute, by waiting the aditional amount at the end of the set

In [35]:
api_req_limit = 60   # number of requests limited per minute
number_of_sets = int(len(cities)/api_req_limit) + 1
city_count = 0   # counter runs through the list cities

# Variables to hold in the dataframe for each city
latitude_lst = []
longitude_lst = []
max_temperature_lst = []
humidity_lst = []
cloud_coverage_lst = []
wind_speed_lst = []
country_lst = []
datetime_lst = []

for six in range(number_of_sets):
    start_time = time.time()   # in seconds
    for cix in range(api_req_limit):
        try:
            city = cities[city_count]
        except IndexError:  # api_req_limit*number_of_sets may exceed len(cities), giving and IndexError
            print("---------------------------")
            print("Data Retrieval Complete")
            print("---------------------------")
            break
        print("Processing Record %d of Set %d | %s" % (cix, six+1, city))
        
        request_url = "https://api.openweathermap.org/data/2.5/weather?q=%s&appid=%s" % (city, weather_api_key)
        
        try:
            response = requests.get(request_url)
            response_json = response.json()
        
            # Append information from json to a list
            latitude_lst.append(response_json["coord"]["lat"])
            longitude_lst.append(response_json["coord"]["lon"])
            max_temperature_lst.append(response_json["main"]["temp_max"])
            humidity_lst.append(response_json["main"]["humidity"])
            cloud_coverage_lst.append(response_json["clouds"]["all"])
            wind_speed_lst.append(response_json["wind"]["speed"])
            country_lst.append(response_json["sys"]["country"])
            datetime_lst.append(response_json["dt"])    
        except:
            print("City not found!")
                    
        city_count += 1
        time.sleep(1)
    
    set_time = time.time()
    # Wait for additional amount until 1 min is reached
    if (set_time - start_time) < 60:
        print("waiting additional %d seconds before moving into the next Set" % (60 - (set_time - start_time)))
        time.sleep(np.ceil(60 - (set_time - start_time)))
        

Processing Record 0 of Set 1 | santa maria
Processing Record 1 of Set 1 | tiksi
Processing Record 2 of Set 1 | rikitea
Processing Record 3 of Set 1 | jamestown
Processing Record 4 of Set 1 | campbell river
Processing Record 5 of Set 1 | ushuaia
Processing Record 6 of Set 1 | karpathos
Processing Record 7 of Set 1 | illoqqortoormiut
City not found!
Processing Record 8 of Set 1 | phan rang
City not found!
Processing Record 9 of Set 1 | mahebourg
Processing Record 10 of Set 1 | tasiilaq
Processing Record 11 of Set 1 | vaini
Processing Record 12 of Set 1 | mataura
Processing Record 13 of Set 1 | sitka
Processing Record 14 of Set 1 | victor harbor
Processing Record 15 of Set 1 | nizhneyansk
City not found!
Processing Record 16 of Set 1 | puerto ayora
Processing Record 17 of Set 1 | chicama
Processing Record 18 of Set 1 | rocha
Processing Record 19 of Set 1 | avarua
Processing Record 20 of Set 1 | castro
Processing Record 21 of Set 1 | tsihombe
City not found!
Processing Record 22 of Set 1 |

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [31]:
response.raise_for_status()

HTTPError: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=illoqqortoormiut&appid=243660c95e5c737c3666b1b5dd4c14c8

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression